Working on the Post-training quantization of the model and the subset of the dataset. 
* Working on the model already trained with the CMS dataset.
* The model has been quantized using the `tf.lite.converter`
* Further, we are working with sample of the dataset(100 for our case). 
* Quantizing the sample of dataset using INT8 quantization


As written on here, we were working on the stuff with the debugging of the model and working out the task. 
* Perform the quantization of the model
* Convert all of the data 
* Put all stuff in a notebook along with the model interface.

## `Task`
Standardize the P_T value before quantizing.

Three P_T plots:

Raw P_T (Number of particles vs P_T (GeV))
P_T after standardization (X-axis should be arbitrary units)
After int8 quantization. Centers at 0 and range should be between -127, 127)
Upon standardization, there could be a few values which could be out of the range of (-127, 127). How to deal with the outliers? One of the methods is to put all of them in the last bin. Are there any other methods available?

Fix bin size does not give resolution. We can lose information if we discard those outliers.

Print the true and predicted P_T values. Should be in the INT8 range.

Put on the distribution if it makes sense.

We need to standradization to whole datasample which is 100 in our case.
* Plot the datasample before and after standradization ??
* Or do we only need to standradize the True P_T?
* What type of output you have right now?

`The dataset contains input features and target features consist of different things like pT, eta, phi etc. so all of those need to be standardized and quantized separately. but you can start with just pT` Task:

1. Since working only on the pT, quntize it after standradization. Further Check the plots before and after quantization.

Quantize the dataset after stnadradization and then do the inference. Check the output plots

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import ROOT 

2024-01-16 16:12:21.824330: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-16 16:12:27.294128: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: Matplotlib requires kiwisolver>=1.3.1; you have 1.0.1

In [2]:
pip install --upgrade matplotlib


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.9 MB/s eta 0:00:00:00:0100:01
DEPRECATION: gosam 2.1.1-4b98559 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of gosam or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: professor2 X.Y.Z has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of professor2 or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sys 
sys.path += ["../../../MLPF/mlpf/particleflow/mlpf/"]
from tfmodel.model_setup import make_model
from tfmodel.utils import parse_config

In [ ]:
config, _ = parse_config("../../../MLPF/mlpf/particleflow/parameters/clic.yaml") #positions on the lxplus


In [ ]:

model = make_model(config, tf.float32)
model.build((1, None, config["dataset"]["num_input_features"]))

In [ ]:
model.summary()

In [ ]:
model.load_weights("weights-96-5.346523.hdf5", skip_mismatch=False, by_name=True)
## These files hosted at https://huggingface.co/jpata/particleflow/tree/clic_clusters_v1.6

In [ ]:
## Reading the dataset
ds_builder = tfds.builder("clic_edm_qq_pf", data_dir = '../../../MLPF/mlpf/tensorflow_datasets/') # Tensorflow datsets positions in the lxplus
dss = ds_builder.as_data_source("test")

In [ ]:
def yield_from_ds():
    for elem in dss:
        yield {"X": elem["X"], "ygen": elem["ygen"], "ycand": elem["ycand"]}

In [ ]:
output_signature = {k: tf.TensorSpec(shape=(None, v.shape[1])) for (k, v) in dss.dataset_info.features.items()}
tf_dataset = tf.data.Dataset.from_generator(yield_from_ds, output_signature=output_signature).take(100).padded_batch(batch_size=10)

In [ ]:
data = list(tfds.as_numpy(tf_dataset))


In [ ]:
Xs = [d["X"] for d in data]
ys = [d["ygen"] for d in data]
ycs= [d["ycand"] for d in data]

In [ ]:
true_pts = []
pred_pts = []

for ibatch in range(len(Xs)):
    ret = model(Xs[ibatch])

    mask_true_particles = ys[ibatch][..., 0]!=0
    
    true_pt = ys[ibatch][mask_true_particles, 2]
    pred_pt = ret["pt"][mask_true_particles][..., 0].numpy()

    true_pts.append(true_pt)
    pred_pts.append(pred_pt)

In [ ]:
true_pt = np.concatenate(true_pts)
pred_pt = np.concatenate(pred_pts)

In [ ]:
plt.hist(pred_pt/true_pt, bins=np.linspace(0,3,100));
plt.yscale("log")

In [ ]:
Task:-
1. Standradize the input(why?)
2. Further quantize it